In [ ]:
import json
from pyld import jsonld
import cmipld
from cmipld.utils.git import get_repo_url
from cmipld.utils.server_tools.offline_patched import LD_server
import os

In [ ]:
# Load a local JSON-LD document
with open('experiment/graph.jsonld', 'r') as f:
    gr = json.load(f)

gr_ctx = gr.get('@context', {})
gr_ctx

In [ ]:
# Resolve each context entry in gr['@context'] via cmipld's loader.
# For URL contexts, we fetch and try to return the term definition for that key;
# if not found, we store the entire fetched @context so you can inspect it.
context_results = {}
if isinstance(gr_ctx, dict):
    for ctx_key, ctx_val in gr_ctx.items():
        # Skip embedded dict contexts — they’re already local definitions
        if isinstance(ctx_val, dict):
            context_results[ctx_key] = ctx_val
            continue
        try:
            loaded = cmipld.loader.loader(ctx_val)
            if isinstance(loaded, dict):
                loaded_ctx = loaded.get('@context', loaded)
                if isinstance(loaded_ctx, dict):
                    context_results[ctx_key] = loaded_ctx.get(ctx_key, loaded_ctx)
                else:
                    context_results[ctx_key] = loaded_ctx
            else:
                context_results[ctx_key] = loaded
        except Exception as e:
            context_results[ctx_key] = f"Error: {e}"
else:
    # If @context is a list/IRI, just store it
    context_results['@context'] = gr_ctx

context_results

In [ ]:
# Expand/compact the local document using its own context
expanded = jsonld.expand(gr)
expanded

# Prefer passing just the context (object/IRI), not the full document
compacted = jsonld.compact(gr, gr_ctx)
compacted

In [ ]:
# Convert to RDF (dataset form) and collect unique IRIs found anywhere in the triples
rdf_dataset = jsonld.to_rdf(gr)
all_ids = set()
for _, triples in rdf_dataset.items():
    for triple in triples:
        for key in ('subject', 'predicate', 'object'):
            term = triple[key]
            if term.get('type') == 'IRI':
                all_ids.add(term['value'])

sorted_all_ids = sorted(all_ids)
for iri in sorted_all_ids:
    print(iri)


In [ ]:
# Collect only linked object IRIs
rdf_dataset2 = jsonld.to_rdf(gr)
linked_ids = set()
for _, triples in rdf_dataset2.items():
    for triple in triples:
        obj = triple['object']
        if obj.get('type') == 'IRI':
            linked_ids.add(obj['value'])

for iri in sorted(linked_ids):
    print(iri)


In [ ]:
# Discover the current repo and reverse mapping to a prefix known by cmipld
repo_url = get_repo_url()
prefix = cmipld.reverse_direct[repo_url]
repo_url, prefix

In [ ]:
# Serve the local repo as a JSON-LD endpoint (handy for offline work)
location = os.getcwd()
local = [(location, cmipld.mapping[prefix], prefix)]
server = LD_server(copy=local, use_ssl=False)
base_url = server.start_server(port=8081)  # may pick a different port if 8081 is busy
base_url

In [ ]:
# Work with a known resource via cmipld’s prefix resolution
# If your project uses a different prefix (e.g., 'mip'), replace 'cmip7' accordingly.
amip_doc_iri = 'cmip7:experiment/amip'
amip_ctx_iri = 'cmip7:experiment/_context_'

expanded_amip = jsonld.expand(amip_doc_iri)
expanded_amip

compacted_amip = jsonld.compact(amip_doc_iri, amip_ctx_iri)
compacted_amip

In [ ]:
# RDF for the amip resource
rdf_amip = jsonld.to_rdf(amip_doc_iri)
rdf_amip

In [ ]:
# High-level fetch using cmipld.get, and inspect dependencies
amip = cmipld.get(amip_doc_iri)
amip

deps = cmipld.depends(amip_doc_iri)
deps